In [1]:
import numpy as np 
from numpy import pi
import matplotlib.pyplot as plt
import sciann as sn 
from sciann_datagenerator import * 

---------------------- SCIANN 0.6.4.2 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://www.sciencedirect.com/science/article/pii/S0045782520307374", 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 

 Need support or would like to contribute, please join sciann`s slack group: 
 +  "https://join.slack.com/t/sciann/shared_invite/zt-ne1f5jlx-k_dY8RGo3ZreDXwz0f~CeA" 
 
TensorFlow Version: 2.4.1 
Python Version: 3.8.5 (default, Sep  4 2020, 02:22:02) 
[Clang 10.0.0 ] 



In [2]:
lmbd = 1.0
mu = 0.5
Q = 4.0

In [3]:
# Neural Network Setup.
dtype='float32'

x = sn.Variable("x", dtype=dtype)
y = sn.Variable("y", dtype=dtype)

Uxy = sn.Functional("Uxy", [x, y], 4*[40], 'tanh')
Vxy = sn.Functional("Vxy", [x, y], 4*[40], 'tanh')

# Sxx = sn.Functional("Sxx", [x, y], 4*[40], 'tanh')
# Syy = sn.Functional("Syy", [x, y], 4*[40], 'tanh')
# Sxy = sn.Functional("Sxy", [x, y], 4*[40], 'tanh')

lmbd = sn.Parameter(np.random.rand(), name='lmbd', inputs=[x,y])
mu = sn.Parameter(np.random.rand(), name='mu', inputs=[x,y])

C11 = (2*mu + lmbd)
C12 = lmbd
C33 = 2*mu

Exx = sn.diff(Uxy, x)
Eyy = sn.diff(Vxy, y)
Exy = (sn.diff(Uxy, y) + sn.diff(Vxy, x))*0.5

# c1 = sn.Tie(Sxx, Exx*C11 + Eyy*C12)
# c2 = sn.Tie(Syy, Eyy*C11 + Exx*C12)
# c3 = sn.Tie(Sxy, Exy*C33)

Sxx= Exx*C11 + Eyy*C12
Syy= Eyy*C11 + Exx*C12
Sxy= Exy*C33


Fx = - 1.*(4*pi**2*sn.cos(2*pi*x)*sn.sin(pi*y) - Q*y**3*pi*sn.cos(pi*x)) \
   - 0.5*(pi**2*sn.cos(2*pi*x)*sn.sin(pi*y) - Q*y**3*pi*sn.cos(pi*x)) \
   - 0.5*8*pi**2*sn.cos(2*pi*x)*sn.sin(pi*y)

Fy = 1.0*(3*Q*y**2*sn.sin(pi*x) - 2*pi**2*sn.cos(pi*y)*sn.sin(2*pi*x)) \
   - 0.5*(2*pi**2*sn.cos(pi*y)*sn.sin(2*pi*x) + (Q*y**4*pi**2*sn.sin(pi*x))/4) \
   + 0.5*6*Q*y**2*sn.sin(pi*x)

Lx = sn.diff(Sxx, x) + sn.diff(Sxy, y) - Fx
Ly = sn.diff(Sxy, x) + sn.diff(Syy, y) - Fy


Du = sn.Data(Uxy)
Dv = sn.Data(Vxy)


targets = [Lx, Ly, Du, Dv]

In [4]:
m = sn.SciModel([x, y], targets)

In [5]:
x_data, y_data = np.meshgrid(np.linspace(0.,1.,20), np.linspace(0., 1., 20))

u_data = np.cos(2*pi*x_data) * np.sin(pi*y_data)
v_data = np.sin(pi*x_data) * Q * y_data**4/4
# syy_data = 2*Q*np.sin(pi*y_data)*(y_data==1.)

Lx_data = np.zeros_like(x_data)
Ly_data = np.zeros_like(y_data)

c1_data = np.zeros_like(y_data)
c2_data = np.zeros_like(y_data)
c3_data = np.zeros_like(y_data)

In [ ]:
h = m.train([x_data, y_data], [Lx_data, Ly_data, u_data, v_data], #syy_data], 
            batch_size=50, epochs=2000, log_parameters={'parameters':[lmbd, mu], 'freq':1},
            adaptive_weights={'method':'NTK', 'freq': 100})


Total samples: 400 
Batch size: 50 
Total batches: 8 


+ NTK evaluation time [s]: 2.8531839847564697 

+ adaptive_weights at epoch 1: [0.13985018048330228, 0.1332919400120442, 1.6680783779223054, 2.0587795015823485]
Epoch 1/2000
8/8 [==============================] - 6s 39ms/step - batch: 3.5000 - size: 50.0000 - loss: 265.6857 - sub_7_loss: 1662.9235 - sub_8_loss: 238.6782 - Uxy_loss: 0.6363 - Vxy_loss: 0.1215
Epoch 2/2000
8/8 [==============================] - 0s 5ms/step - batch: 3.5000 - size: 50.0000 - loss: 220.4545 - sub_7_loss: 1373.1880 - sub_8_loss: 199.5286 - Uxy_loss: 0.7468 - Vxy_loss: 0.2781
Epoch 3/2000
8/8 [==============================] - 0s 5ms/step - batch: 3.5000 - size: 50.0000 - loss: 190.2717 - sub_7_loss: 1194.2499 - sub_8_loss: 155.7637 - Uxy_loss: 1.2082 - Vxy_loss: 0.2323
Epoch 4/2000
8/8 [==============================] - 0s 5ms/step - batch: 3.5000 - size: 50.0000 - loss: 169.4628 - sub_7_loss: 1054.1567 - sub_8_loss: 151.8634 - Uxy_loss: 0.6948 - Vxy_lo

8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 3.4676 - sub_7_loss: 8.5876 - sub_8_loss: 2.8764 - Uxy_loss: 0.8242 - Vxy_loss: 0.2469
Epoch 42/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 3.3085 - sub_7_loss: 8.2590 - sub_8_loss: 2.7216 - Uxy_loss: 0.7731 - Vxy_loss: 0.2434 
Epoch 43/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 3.1545 - sub_7_loss: 8.0803 - sub_8_loss: 2.5204 - Uxy_loss: 0.7177 - Vxy_loss: 0.2387
Epoch 44/2000
8/8 [==============================] - 0s 5ms/step - batch: 3.5000 - size: 50.0000 - loss: 2.9857 - sub_7_loss: 7.3181 - sub_8_loss: 2.6138 - Uxy_loss: 0.6658 - Vxy_loss: 0.2444
Epoch 45/2000
8/8 [==============================] - 0s 5ms/step - batch: 3.5000 - size: 50.0000 - loss: 2.8487 - sub_7_loss: 7.0312 - sub_8_loss: 2.5055 - Uxy_loss: 0.6188 - Vxy_loss: 0.2425
Epoch 46/2000
8/8 [==============================] - 

8/8 [==============================] - 0s 5ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.7437 - sub_7_loss: 2.1044 - sub_8_loss: 0.6239 - Uxy_loss: 0.0614 - Vxy_loss: 0.1282
Epoch 84/2000
8/8 [==============================] - 0s 5ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.7518 - sub_7_loss: 2.1189 - sub_8_loss: 0.6529 - Uxy_loss: 0.0611 - Vxy_loss: 0.1295
Epoch 85/2000
8/8 [==============================] - 0s 5ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.7264 - sub_7_loss: 2.0321 - sub_8_loss: 0.6406 - Uxy_loss: 0.0589 - Vxy_loss: 0.1256
Epoch 86/2000
8/8 [==============================] - 0s 5ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.7623 - sub_7_loss: 2.3328 - sub_8_loss: 0.5911 - Uxy_loss: 0.0608 - Vxy_loss: 0.1243
Epoch 87/2000
8/8 [==============================] - 0s 5ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.9968 - sub_7_loss: 3.9206 - sub_8_loss: 0.7115 - Uxy_loss: 0.0612 - Vxy_loss: 0.1223
Epoch 88/2000
8/8 [==============================] - 0

8/8 [==============================] - 0s 5ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.2238 - sub_7_loss: 1.5399 - sub_8_loss: 0.7731 - Uxy_loss: 0.0339 - Vxy_loss: 0.0653
Epoch 125/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.2222 - sub_7_loss: 1.6066 - sub_8_loss: 0.7553 - Uxy_loss: 0.0336 - Vxy_loss: 0.0643
Epoch 126/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.2202 - sub_7_loss: 1.5259 - sub_8_loss: 0.7688 - Uxy_loss: 0.0337 - Vxy_loss: 0.0634
Epoch 127/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.2197 - sub_7_loss: 1.6000 - sub_8_loss: 0.7499 - Uxy_loss: 0.0333 - Vxy_loss: 0.0633
Epoch 128/2000
8/8 [==============================] - 0s 5ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.2186 - sub_7_loss: 1.5781 - sub_8_loss: 0.7741 - Uxy_loss: 0.0332 - Vxy_loss: 0.0626
Epoch 129/2000
8/8 [==============================

8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.1785 - sub_7_loss: 1.5411 - sub_8_loss: 0.7658 - Uxy_loss: 0.0279 - Vxy_loss: 0.0456
Epoch 167/2000
8/8 [==============================] - 0s 3ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.1797 - sub_7_loss: 1.4218 - sub_8_loss: 0.7745 - Uxy_loss: 0.0281 - Vxy_loss: 0.0472
Epoch 168/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.1779 - sub_7_loss: 1.3695 - sub_8_loss: 0.7801 - Uxy_loss: 0.0284 - Vxy_loss: 0.0460
Epoch 169/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.1774 - sub_7_loss: 1.3922 - sub_8_loss: 0.7514 - Uxy_loss: 0.0280 - Vxy_loss: 0.0465
Epoch 170/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.1756 - sub_7_loss: 1.4304 - sub_8_loss: 0.8028 - Uxy_loss: 0.0277 - Vxy_loss: 0.0447
Epoch 171/2000
8/8 [==============================

...

8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0038 - sub_7_loss: 0.4262 - sub_8_loss: 0.0977 - Uxy_loss: 2.6196e-04 - Vxy_loss: 9.5881e-04
Epoch 953/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0043 - sub_7_loss: 0.4421 - sub_8_loss: 0.1087 - Uxy_loss: 2.7272e-04 - Vxy_loss: 0.0012
Epoch 954/2000
8/8 [==============================] - 0s 40ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0045 - sub_7_loss: 0.4739 - sub_8_loss: 0.0979 - Uxy_loss: 3.3631e-04 - Vxy_loss: 0.0013
Epoch 955/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0053 - sub_7_loss: 0.4980 - sub_8_loss: 0.1171 - Uxy_loss: 6.1420e-04 - Vxy_loss: 0.0012
Epoch 956/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0053 - sub_7_loss: 0.4349 - sub_8_loss: 0.0970 - Uxy_loss: 4.4661e-04 - Vxy_loss: 0.0016
Epoch 957/2000
8/8 [=====

8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0043 - sub_7_loss: 0.4799 - sub_8_loss: 0.0848 - Uxy_loss: 4.4949e-04 - Vxy_loss: 0.0010
Epoch 994/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0038 - sub_7_loss: 0.4090 - sub_8_loss: 0.1146 - Uxy_loss: 3.1908e-04 - Vxy_loss: 8.3729e-04
Epoch 995/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0042 - sub_7_loss: 0.5239 - sub_8_loss: 0.1104 - Uxy_loss: 3.1049e-04 - Vxy_loss: 9.6300e-04
Epoch 996/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0037 - sub_7_loss: 0.4672 - sub_8_loss: 0.0851 - Uxy_loss: 2.6866e-04 - Vxy_loss: 9.5063e-04
Epoch 997/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0037 - sub_7_loss: 0.4327 - sub_8_loss: 0.0756 - Uxy_loss: 2.8840e-04 - Vxy_loss: 9.9480e-04
Epoch 998/2000

8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0042 - sub_7_loss: 0.3355 - sub_8_loss: 0.1128 - Uxy_loss: 3.3572e-04 - Vxy_loss: 0.0011    
Epoch 1033/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0033 - sub_7_loss: 0.3413 - sub_8_loss: 0.0685 - Uxy_loss: 2.8198e-04 - Vxy_loss: 8.6457e-04
Epoch 1034/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0036 - sub_7_loss: 0.3489 - sub_8_loss: 0.0603 - Uxy_loss: 3.6106e-04 - Vxy_loss: 9.6301e-04
Epoch 1035/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0032 - sub_7_loss: 0.3557 - sub_8_loss: 0.0588 - Uxy_loss: 3.3033e-04 - Vxy_loss: 7.7395e-04
Epoch 1036/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0035 - sub_7_loss: 0.4009 - sub_8_loss: 0.0724 - Uxy_loss: 3.8090e-04 - Vxy_loss: 7.2520e-04
Epoch 

8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0039 - sub_7_loss: 0.3137 - sub_8_loss: 0.0663 - Uxy_loss: 4.0805e-04 - Vxy_loss: 0.0011
Epoch 1074/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0042 - sub_7_loss: 0.4555 - sub_8_loss: 0.0965 - Uxy_loss: 3.1671e-04 - Vxy_loss: 0.0010    
Epoch 1075/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0059 - sub_7_loss: 0.4784 - sub_8_loss: 0.1196 - Uxy_loss: 3.4580e-04 - Vxy_loss: 0.0019    
Epoch 1076/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0035 - sub_7_loss: 0.2697 - sub_8_loss: 0.0770 - Uxy_loss: 3.5025e-04 - Vxy_loss: 9.2182e-04
Epoch 1077/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0036 - sub_7_loss: 0.2858 - sub_8_loss: 0.0690 - Uxy_loss: 2.9030e-04 - Vxy_loss: 0.0011
Epoch 1078/200

8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0036 - sub_7_loss: 0.2714 - sub_8_loss: 0.0587 - Uxy_loss: 5.5785e-04 - Vxy_loss: 7.5507e-04
Epoch 1114/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0031 - sub_7_loss: 0.3443 - sub_8_loss: 0.0497 - Uxy_loss: 3.5660e-04 - Vxy_loss: 7.0161e-04
Epoch 1115/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0031 - sub_7_loss: 0.3093 - sub_8_loss: 0.0586 - Uxy_loss: 3.1157e-04 - Vxy_loss: 7.6161e-04
Epoch 1116/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0028 - sub_7_loss: 0.2399 - sub_8_loss: 0.0559 - Uxy_loss: 2.5974e-04 - Vxy_loss: 7.3094e-04
Epoch 1117/2000
8/8 [==============================] - 0s 4ms/step - batch: 3.5000 - size: 50.0000 - loss: 0.0029 - sub_7_loss: 0.3191 - sub_8_loss: 0.0531 - Uxy_loss: 3.1252e-04 - Vxy_loss: 6.8604e-04
Epoch 

In [ ]:
h.history.keys()

In [ ]:
plt.plot(h.history['parameter_lmbd'], label='lmbd')
plt.plot(h.history['parameter_mu'], label='mu')
plt.legend()

In [ ]:
x_test, y_test = np.meshgrid(np.linspace(0,1,100), np.linspace(0,1,100))

In [ ]:
test_data = {
    'u': Uxy.eval([x_test, y_test]),
    'v': Vxy.eval([x_test, y_test]),
    'sxx': Sxx.eval([x_test, y_test]),
    'syy': Syy.eval([x_test, y_test]),
    'sxy': Sxy.eval([x_test, y_test])
}

In [ ]:
def cust_plot(ax, val, label):
    im = ax.pcolor(x_test, y_test, val, cmap='seismic', shading='auto', 
                   vmin=-np.abs(val).max(), vmax=np.abs(val).max())
    ax.set_title(label)
    plt.colorbar(im, ax=ax)
    
fig, ax = plt.subplots(1,5,figsize=(15,3))
for i, (key, val) in enumerate(test_data.items()):
    cust_plot(ax[i], val, key)
    
plt.tight_layout()
plt.show()